<a href="https://colab.research.google.com/github/ajitonelsonn/TELE_Feedback/blob/main/Llama_Chatbot_with_Sentiment_Analysis_Integration_TL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TELE Feedback**

In [ ]:
!pip install huggingface_hub gradio git+https://github.com/huggingface/transformers sqlalchemy pandas matplotlib torch

In [ ]:
from huggingface_hub import login
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import torch
import matplotlib.pyplot as plt
from collections import Counter
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

In [ ]:
# Hugging Face login (you need to provide your token)
login(token="")


In [ ]:
# Database setup
Base = declarative_base()
engine = create_engine('sqlite:///telecom_support.db', echo=False)
Session = sessionmaker(bind=engine)

class Feedback(Base):
    __tablename__ = 'feedback'
    id = Column(Integer, primary_key=True)
    category = Column(String)
    original_text = Column(String)
    sentiment = Column(String)
    response = Column(String)

Base.metadata.create_all(engine)

In [ ]:
# Load Llama 2 model
model_id = "NousResearch/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
# Updated categories for a telecom company
CATEGORIES = ["Mobile Network", "Mobile Package", "Mosan Emoney", "Recharge"]

In [ ]:
def generate_response(prompt, max_length=500):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1, temperature=0.7)

    # Decode the generated response
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
def process_feedback(message, category):
    prompt = f"""
    You are a professional customer service assistant for a telecom company. A customer has submitted the following feedback in the category of {category}:

    "{message}"

    Please respond with:
    1. A clear identification of the sentiment (positive, negative, or neutral).
    2. A concise, empathetic, and professional response addressing the customer's concern.
    3. If appropriate, include a solution or next steps for the customer.

    Format the output as:
    Sentiment: [positive/negative/neutral]
    Response: [Your customer response]

    No extra information or format explanations should be included.
    """

    # Generate response from the model
    response = generate_response(prompt, max_length=500)

    return response

In [ ]:
def chatbot(message, category):
    if not message.strip():
        return "Error: Please provide feedback text."
    if not category:
        return "Error: Please select a category."

    try:
        # Process the feedback with the category
        result = process_feedback(message, category)

        # Parse the result
        lines = result.split('\n')
        parsed_result = {}
        for line in lines:
            if ':' in line:
                key, value = line.split(':', 1)
                parsed_result[key.strip()] = value.strip()

        # Extract sentiment and response
        sentiment = parsed_result.get('Sentiment', 'Unknown')
        response = parsed_result.get('Response', '')

        # If no response is generated, use a fallback
        if not response:
            fallback_prompt = f"Provide a brief, helpful response to a customer's feedback about {category}: '{message}'"
            response = generate_response(fallback_prompt, max_length=150)

        # Save the feedback to the database
        session = Session()
        feedback = Feedback(
            category=category,
            original_text=message,
            sentiment=sentiment,
            response=response
        )
        session.add(feedback)
        session.commit()

        return response

    except Exception as e:
        error_prompt = f"Briefly apologize and offer assistance for an error in processing feedback about {category}: '{message}'"
        error_response = generate_response(error_prompt, max_length=100)
        return error_response

In [ ]:
# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Telecommunication Customer Support Chatbot")

    category = gr.Dropdown(choices=CATEGORIES, label="Select Feedback Category")
    message = gr.Textbox(lines=3, placeholder="Type your feedback here...", label="Your Feedback")

    # Use TextArea for better long-text handling
    output = gr.TextArea(label="Chatbot Response", lines=10)

    submit_btn = gr.Button("Submit Feedback")
    submit_btn.click(fn=chatbot, inputs=[message, category], outputs=output)

    gr.Examples(
        examples=[
            ["Mobile Network", "The network is down in my area for days."],
            ["Mobile Package", "I'm happy with the new data package!"],
            ["Mosan Emoney", "I have an issue with my last Mosan Emoney transfer."],
            ["Recharge", "I need help with my recent recharge transaction."]
        ],
        inputs=[category, message]
    )

In [ ]:
demo.launch()

In [ ]:
# Function to plot the data
def plot_sentiment_distribution():
    # Query the database to retrieve the feedback data
    session = Session()
    feedback_data = session.query(Feedback).all()

    # Processing data: count sentiments per category
    category_sentiment_count = {}

    for feedback in feedback_data:
        category = feedback.category
        sentiment = feedback.sentiment.lower()  # Convert sentiment to lowercase for consistency

        if category not in category_sentiment_count:
            category_sentiment_count[category] = Counter()

        category_sentiment_count[category][sentiment] += 1

    session.close()

    # Prepare data for plotting
    categories = list(category_sentiment_count.keys())
    positive_counts = [category_sentiment_count[cat].get('positive', 0) for cat in categories]
    negative_counts = [category_sentiment_count[cat].get('negative', 0) for cat in categories]
    neutral_counts = [category_sentiment_count[cat].get('neutral', 0) for cat in categories]

    # Plotting the data with adjusted bar width and position
    x = range(len(categories))  # x-coordinates
    bar_width = 0.2  # Adjust the bar width

    plt.figure(figsize=(10, 6))

    # Plotting positive, negative, and neutral sentiment bars for each category
    plt.bar([i - bar_width for i in x], positive_counts, width=bar_width, label='Positive', align='center')
    plt.bar(x, negative_counts, width=bar_width, label='Negative', align='center')
    plt.bar([i + bar_width for i in x], neutral_counts, width=bar_width, label='Neutral', align='center')

    # Customizing the plot
    plt.xlabel('Feedback Categories')
    plt.ylabel('Number of Feedback')
    plt.title('Sentiment Distribution by Feedback Categories')
    plt.xticks(x, categories, rotation=45)
    plt.legend()

    # Save the plot as an image
    plt.tight_layout()
    plt.savefig('sentiment_distribution.png')  # Save the plot as an image
    plt.close()  # Close the plot to avoid display issues

    # Return the image path for Gradio to display
    return 'sentiment_distribution.png'

# Gradio interface
interface = gr.Interface(fn=plot_sentiment_distribution, inputs=[], outputs="image", title="Telecom Support Feedback Sentiment")

# Launch the interface
interface.launch()